## Машинное обучение, ВМК МГУ

## Практическое задание 2

### Общая информация
Дата выдачи: 9 октября 2019

Максимальная оценка: 10 баллов + 1 бонусный балл

Мягкий дедлайн: 23:59MSK 23 октября (за каждый день просрочки снимается 1 балл)

Жесткий дедлайн: 23:59MSK 30 октября.

### О задании

В этом задании вы:
- Познакомитесь с методом решения задачи регрессии на основе метода ближайших соседей.
- Реализуете алгоритм kNN для задачи регрессии.
- Изучите методы работы с категориальными и текстовыми переменными.


### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов.

Сдавать задание после указанного срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.


### Формат сдачи
Для сдачи задания переименуйте получившийся файл *.ipynb в соответствии со следующим форматом: homework-practice-02-Username.ipynb, где Username — ваша фамилия и имя на латинице именно в таком порядке (например, homework-practice-02-ivanov.ipynb).

Далее отправьте этот файл на anytask в соответсвующий раздел.

In [269]:
import numpy as np
import pandas as pd

Все эксперименты в этой лабораторной работе предлагается проводить на данных соревнования New York City Airbnb Open Data: https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data#AB_NYC_2019.csv

В данной задаче предлагается предсказать цену на съем квартиры в зависимости от её параметров.

In [270]:
data = pd.read_csv('AB_NYC_2019.csv')
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [271]:
data.shape

(48895, 16)

In [272]:
# число значений у признаков
for col_name in data.columns:
    print(col_name, len(data[col_name].unique()))

id 48895
name 47906
host_id 37457
host_name 11453
neighbourhood_group 5
neighbourhood 221
latitude 19048
longitude 14718
room_type 3
price 674
minimum_nights 109
number_of_reviews 394
last_review 1765
reviews_per_month 938
calculated_host_listings_count 47
availability_365 366


In [273]:
data.isna().sum()

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

Как видите, в данных есть пропуски. Не забудьте обработать их

In [274]:
# Ваш код здесь (ノ°∀°)ノ⌒･*:.｡. .｡.:*･゜ﾟ･*☆
numberCols = data.select_dtypes(include=[np.number]).columns.tolist()
numberCols.remove('id')
numberCols.remove('price')

categoricalCols = data.select_dtypes(include=[object]).columns.tolist()

data[numberCols] = data[numberCols].fillna(0)
data[categoricalCols] = data[categoricalCols].fillna('')

Разобъем данные на обучение и контроль.

In [275]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=['price']), data[['price']],
                                                    test_size=0.3, random_state=241)

## Часть 1: Алгоритм kNN в задаче регрессии

<b> 1.1 (1.5 балла) </b>
Реализуйте класс `KNNRegressor`, который используя метод k ближайших соседей решает задачу регрессии. Для решение данной задачи, необходимо найти $N_k$ - k соседей, и после использовать значения их целевых переменных для предсказания:
\begin{align}
y = \frac{1}{k}\sum_{n \in N_k}w_n y_n,
\end{align}

где $w_n$ - вес каждого соседа. 

При этом `KNNRegressor` может работать в 2 режимах:
 - $uniform$ - ближайшие соседи учитываются с одинаковыми весами.
 - $distance$ - вес ближайших соседей зависит от расстояния
 
Сигнатуру методов при желании можно менять.

In [292]:
from typing import Union, Callable, Iterable, Optional
# from sklearn.neighbors import NearestNeighbors
# from sklearn.neighbors import KNeighborsRegressor

class KNNRegressor:
    def __init__(self, n_neighbors: int, metric: Union[str, Callable], mode: str = 'uniform'):
        """
        params:
            n_neighbors: number of neighbors
            metric: metric to use for distance computation
            mode: 'uniform' or 'distance'
            'uniform' - all points in each neighborhood are weighted equally
            'distance' - weight points by the inverse of their distance
        """
#         self.__nn = NearestNeighbors(n_neighbors = n_neighbors, metric = metric)
#         self.__nnTest = KNeighborsRegressor(n_neighbors = n_neighbors, metric = metric, weights = mode)
        self.__metric = metric
        self.__mode = mode
        self.__n_neighbors = n_neighbors

    def fit(self, X: np.array, y: np.array) -> None:
        """
            X: data
            y: labels
        """
        # Ваш код здесь
        self.__X = X
        self.__y = y
        
#         self.__nn.fit(X, y)
#         self.__nnTest.fit(X, y)
        
    def euclidean_distance(self, X, Y):
        d = len(X[0])
        res = np.zeros(len(X)*len(Y)).reshape(len(X), len(Y))
        for x in range(len(X)):
            for y in range(len(Y)):
                sum = 0
                for f in range(len(Y[y])):
                    sum += (X[x][f] - Y[y][f])**2
                res[x][y] = np.sqrt(sum)
        return res;

    def cosine_distance(self, X, Y):
        sumyy = (Y**2).sum(1)
        sumxx = (X**2).sum(1, keepdims=1)
        sumxy = X.dot(Y.T)
        return 1 - (sumxy/np.sqrt(sumxx))/np.sqrt(sumyy)
    
    def overlap(self, X, Y):
        res = []
        for row in range(len(X)):
            distRow = []
            for col in range(len(X[row])):
                if (X[row][col] == Y[row][col]):
                    distRow.append(0)
                else:
                    distRow.append(1)
            res.append(distRow)
        return np.array(res)
    
    def find_kneighbors(self, X, return_distance = True):
        res = []
        dists = []
        
        if (self.__metric == 'euclidean'):
            dsModRes = self.euclidean_distance(X, self.__X)
        elif (self.__metric == 'cosine'):
            dsModRes = self.cosine_distance(X, self.__X)
        elif (self.__metric == 'overlap'):
            dsModRes = self.overlap(X, self.__X)
        else:
            raise Exception("Unknown 'metric' param value")

        for i in range(len(dsModRes)):
            tmpDists = np.argsort(dsModRes[i])[:self.__n_neighbors]
            res.append(dsModRes[i][tmpDists])
            if (return_distance):
                dists.append(tmpDists)

        if (return_distance):
            return (res, dists)
        else:
            return (res)
        
    def predict(self, X: np.array, n_neighbors: Optional[int] = None) -> np.array:
        """
            X: data
            n_neighbors: number of neighbors
        """
        # Ваш код здесь
#         distances, indices = self.__nn.kneighbors(X)
#         distances, indices = self.__nnTest.kneighbors(X)
        distances, indices = self.find_kneighbors(X)

        if (self.__mode == 'uniform'):
            res = self.__y[indices].mean(axis=1)
        else: 
            res = []
            for row in range(len(indices)):
                predictedVal = 0
                weightsSum = 0
                for col in range(len(indices[row])):
                    w = 1 / distances[row][col]
                    weightsSum = weightsSum + w
                    predictedVal = predictedVal + w * self.__y[indices[row][col]]
                res.append(predictedVal / weightsSum)
            res = np.array(res)
#         return self.__nnTest.predict(X)
        return res
            

In [293]:
X_train1 = X_train[0:1000][numberCols].to_numpy()
X_test1 = X_test[0:1000][numberCols].to_numpy()
y_train1 = y_train[0:1000].to_numpy()[:,0]
y_test1 = y_test[0:1000].to_numpy()[:,0]

knnr = KNNRegressor(n_neighbors = 3, metric = 'euclidean', mode = 'distance')
knnr.fit(X_train1, y_train1)
res = knnr.predict(X_test1)

print(res)

[ 90.29251629 397.87046594 353.79719069 159.72786592 143.31365627
  72.736916   127.40247415 162.76308321 341.17736679  79.71418919
 340.10600726  94.72574994  90.89636432  67.26622542 109.75617273
  90.01230259  81.24088082 123.59205132 198.3509165   86.19272515
 219.15969862  96.54049984 234.52202703 108.81511737  81.90794333
 115.42292938  52.41721869  61.1710198  106.96531463 120.78855289
 257.64086499 103.13875653  52.10170039  78.33080373  92.5181504
 271.00148252  53.0881226   98.25416924 154.13598334 272.39533882
  85.16307246  67.55956254  97.286871   293.88664741  99.72745368
 144.85030814 180.89048855  66.06239047 102.51315268  72.5035415
 498.4179091  150.32369323 105.39241672 156.63943436 155.2386555
 124.65423332  61.2169752  108.08578947 150.05476286  58.99330874
  65.27604551  43.14825494  50.84214045 122.02300734  68.99662844
 105.67330905  87.64381876  90.57881448 100.62983982 228.78632221
 139.90224706  64.98356114  86.75824543  83.5168715  220.71820173
 142.15932238

## Часть 2: Категориальные признаки

<b>2.1 (1 балл)</b>
Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на [третьем семинаре](https://github.com/mmp-mmro-team/mmp_mmro_fall_2019/blob/master/lecture-notes/Sem03_knn.pdf). Не забудьте, что KNNRegressor должен уметь работать с этими функциями расстояния. Как вариант, можно реализовать метрики как [user-defined distance](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html).

In [176]:
def overlap(x, z):
    # Ваш код здесь
    dist = 0
    for i in range(len(x)):
        if (x[i] != z[i]):
            dist = 1
            break
    return dist

def flattened_overlap(x, z):
    # Ваш код здесь
    pass

def log_overlap(x, z):
    # Ваш код здесь
    pass

In [149]:
X_train1 = X_train[0:1000][categoricalCols].to_numpy()
X_test1 = X_test[0:1000][categoricalCols].to_numpy()
y_train1 = y_train[0:1000].to_numpy()[:,0]
y_test1 = y_test[0:1000].to_numpy()[:,0]

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
X_train1[:, 0] = labelencoder.fit_transform(X_train1[:, 0])
X_train1[:, 1] = labelencoder.fit_transform(X_train1[:, 1])
X_train1[:, 2] = labelencoder.fit_transform(X_train1[:, 2])
X_train1[:, 3] = labelencoder.fit_transform(X_train1[:, 3])
X_train1[:, 4] = labelencoder.fit_transform(X_train1[:, 4])
X_train1[:, 5] = labelencoder.fit_transform(X_train1[:, 5])

X_test1[:, 0] = labelencoder.fit_transform(X_test1[:, 0])
X_test1[:, 1] = labelencoder.fit_transform(X_test1[:, 1])
X_test1[:, 2] = labelencoder.fit_transform(X_test1[:, 2])
X_test1[:, 3] = labelencoder.fit_transform(X_test1[:, 3])
X_test1[:, 4] = labelencoder.fit_transform(X_test1[:, 4])
X_test1[:, 5] = labelencoder.fit_transform(X_test1[:, 5])


knnr = KNNRegressor(n_neighbors = 3, metric = 'euclidean', mode = 'uniform')
knnr.fit(X_train1, y_train1)
res = knnr.predict(X_test1)

print(res)

[131.33333333  87.         314.33333333  98.66666667 124.33333333
 103.33333333  80.          51.66666667 249.66666667 408.33333333
 131.66666667  69.33333333  81.33333333 102.66666667 260.
  83.          96.33333333 161.66666667 111.66666667  81.66666667
 266.66666667  85.66666667 111.          70.33333333  90.66666667
 102.66666667 298.          91.33333333  82.33333333 436.66666667
 103.33333333 408.         115.33333333 206.66666667 253.33333333
  96.66666667 135.         196.66666667 107.66666667 169.33333333
  88.          71.66666667  87.66666667 164.66666667 178.33333333
 188.         183.         277.66666667 110.          71.66666667
 113.33333333 112.33333333  91.33333333  99.33333333 190.33333333
  98.33333333  84.33333333  75.         107.33333333 198.
  88.          79.33333333 168.          98.33333333  95.
  92.66666667  91.66666667  76.         176.66666667 140.33333333
  50.         104.33333333  95.         206.         172.33333333
 115.         100.         130.   

<b>2.2 (1 балл)</b> Найдите все категориальные признаки в данных. Подсчитайте для каждой из метрик качество на тестовой выборке `X_test` при числе соседей $k = 10$. Качество измеряйте с помощью RMSE.

Какая функция расстояния оказалась лучшей? Почему?

In [207]:
# Ваш код здесь
categoricalCols = numberCols = X_test.select_dtypes(include=[object]).columns.tolist()
categoricalCols

['name',
 'host_name',
 'neighbourhood_group',
 'neighbourhood',
 'room_type',
 'last_review']

<b>2.3 (1 балл) бонус</b> Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какого удалось достичь уровня качества?

In [ ]:
# Ваш код здесь

<b>2.4 (2.5 балла)</b> Отойдем ненадолго от задачи регрессии и перейдём к задаче классификации: будем определять, являеться ли квартира дорогой $(target = 1)$ или дешевой $(target = 0)$. Будем считать дорогими квариры, цена которых выше среднего.

In [ ]:
data['target'] = (data.price > data.price.mean()).astype(int)

Реализуйте счетчики, которые заменят категориальные признаки на вещественные.

А именно, для каждого категориального признака $f_j(x)$ необходимо сделать следующее:
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
\begin{align}
counts_j(c) = \sum_{i=1}^l [f_j(x_i) = c]
\end{align}
2. Число `successes` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
\begin{align}
successes_j(c) = \sum_{i=1}^l[f_j(x_i) = c][y_i = +1].
\end{align}
3. Сглаженное отношение двух предыдущих величин:
\begin{align}
p_j(c) = \frac{successes_j(c) + a}{counts_j(c) + b},
\end{align}

где $a$ и $b$ - априорные счетчики (например, a = 1, b = 2).

In [ ]:
def counters(x):
    """
    params:
        x: value on categorical feature for N objects
    returns: vector of length N
    """
    # Ваш код здесь
    pass

Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `successes` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанный по всей обучающей выборке. Реализуйте и такой вариант. Достаточно взять $n = 3$.

In [ ]:
def fold_counters(x):
    """
    params:
        x: value on categorical feature for N objects
    returns: vector of length N
    """
    # Ваш код здесь
    pass

Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

In [ ]:
# Ваш код здесь

<b>2.5 (1 балл)</b> Вернемся к задаче регрессии. Утверждается, что для задачи регрессии можно также сделать преобразование категориальных признаков в действительные числа. Для этого достаточно для каждого значения признака $f_j$ вычислить:
\begin{align}
p_j(c) = g(T_i | f_j(x_i) = c),
\end{align}

где $T_i$ - значения целевой переменной объекта $x_i$. Функция $g$ - среднее (mean) или среднеквадратичное отклонение (std).

Закодируйте категориальные признаки обоими способами и найдите значение RMSE. Используйте евклидову метрику для поиска ближайших соседей. Для какой функции $g$ значение RMSE лучше? Почему?

In [ ]:
# Ваш код здесь

## Часть 3: Текстовые признаки

<b>3.1 (2 балла)</b> Перейдем от категориальным признаков к текстовым. Рассмотрим 2 способа преобразования текста в действительные числа:
- Мешок слов (Bag of Words)
- TF-IDF

[Здесь](https://scikit-learn.org/stable/modules/feature_extraction.html) вы можете прочитать про их применение в Питоне.

Сравните оба способа на задаче регресси. Какую лучше метрику использовать: евклидову или косинусную меру? Постройте графики зависимости качества решения задачи от способа преобразования, метрики и количества соседей. Мера качества - RMSE.

Объясните полученные результаты.

Перед преобразованием не забудьте уменьшить размер словаря. Например, это можно сделать за счет приведения всех слов к одному регистру и удаления [стопслов](https://en.wikipedia.org/wiki/Stop_words) (артиклей, предлогов, союзов).

In [ ]:
# Ваш код здесь

<b>3.2 (1 балл)</b> Используя все доступные признаки, решите задачу регрессии. Для категориальных и текстовых признаков выберите лучшие преобразования. Повлияло ли добавление количественного признака на метрику качества?

In [7]:
# Ваш код здесь

## Часть 4: Выводы


In [6]:
# Ваши выводы здесь (ノ°∀°)ノ⌒･*:.｡. .｡.:*･゜ﾟ･*☆